# Multitask-Zahlenerkennung mit Transfer Learning
Dieses Notebook lädt handgeschriebene Ziffern (0–9) von drei verschiedenen Personen und trainiert ein Modell, das sowohl die Ziffer als auch die schreibende Person erkennt.

In [ ]:
import os # Import aller benötigten Bibliotheken für das Modell, das Training und die Datenverarbeitung

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
# Eigene Dataset-Klasse zur Verarbeitung der gespeicherten .npy-Bilder
# Extrahiert aus Dateinamen die Ziffer und die zugehörige Person
class DigitPersonDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []

        # Manuelle Zuordnung der Namen zu Klassen-IDs
        self.person_to_id = {'Tim': 0, 'Thanadon': 1, 'Nils': 2}

        # Alle .npy-Dateien im Verzeichnis analysieren und passende Samples sammeln
        for fname in os.listdir(data_dir):
            if fname.endswith(".npy"):
                parts = fname.split("_")
                if len(parts) >= 3:
                    person = parts[0]
                    digit = int(parts[1])
                    person_id = self.person_to_id.get(person, -1)
                    if person_id >= 0:
                        path = os.path.join(data_dir, fname)
                        self.samples.append((path, digit, person_id))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # Bild und Labels laden
        path, digit, person_id = self.samples[idx]
        img_array = np.load(path)

        # Fehlerbehandlung: sicherstellen, dass das Bild richtig formatiert ist
        if img_array.dtype != np.uint8:
            img_array = (img_array * 255 / np.max(img_array)).astype(np.uint8)
        if img_array.ndim > 2:
            img_array = img_array.squeeze()

        img = Image.fromarray(img_array).convert("L")

        # Optional: Bildtransformation anwenden
        if self.transform:
            img = self.transform(img)

        return img, digit, person_id


In [ ]:
# Definition eines Multi-Task CNNs basierend auf ResNet18
# Modell gibt gleichzeitig eine Vorhersage für Ziffer (0–9) und Person (0–2) aus
class MultiTaskResNet(nn.Module):
    def __init__(self):
        super(MultiTaskResNet, self).__init__()
        base = models.resnet18(pretrained=True)  # Vortrainiertes ResNet als Basis

        # Ersetze den ersten Layer, um Graustufenbilder zu unterstützen (1 Kanal)
        base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Entferne den letzten Fully-Connected-Layer (wird durch zwei Köpfe ersetzt)
        self.backbone = nn.Sequential(*list(base.children())[:-1])

        # Klassifikationskopf für Ziffern (10 Klassen)
        self.head_digit = nn.Linear(base.fc.in_features, 10)

        # Klassifikationskopf für Personen (3 Klassen)
        self.head_person = nn.Linear(base.fc.in_features, 3)

    def forward(self, x):
        # Durch das ResNet-Backbone
        x = self.backbone(x)
        x = x.view(x.size(0), -1)  # Flatten
        # Zwei parallele Vorhersagen
        return self.head_digit(x), self.head_person(x)


In [ ]:
# Transformationen für Trainingsbilder: Resize, zufällige Rotation und Translation, Umwandlung in Tensor
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

# Pfad zum Verzeichnis mit den .npy-Bildern
data_path = "C:/dhbw/6. semester/advanced ml/Zahlenerkennung-main/data/processed"  # ggf. anpassen

# Datensatz laden mit definierter Transformation
dataset = DigitPersonDataset(data_path, transform=transform)

# Aufteilung in Training (80 %) und Validierung (20 %)
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_data, val_data = random_split(dataset, [train_len, val_len])

# Erstellen von DataLoadern für Training und Validierung
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

In [ ]:
# Auswahl des Geräts: GPU verwenden, wenn verfügbar, sonst CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modell initialisieren und auf das Gerät verschieben
model = MultiTaskResNet().to(device)

# Verlustfunktion (gemeinsam für Ziffer und Person)
criterion = nn.CrossEntropyLoss()

# Optimierer: Adam mit Lernrate 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# Überprüfung der NumPy-Version (optional)
import numpy
import numpy as np
print(np.__version__)

# Training über 23 Epochen
for epoch in range(23):
    model.train()
    total_loss = 0

    # Mini-Batch-Training über den gesamten Trainingsdatensatz
    for images, labels_digit, labels_person in train_loader:
        images = images.to(device)
        labels_digit = labels_digit.to(device)
        labels_person = labels_person.to(device)

        # Vorwärtsdurchlauf und getrennte Verluste berechnen
        out_digit, out_person = model(images)
        loss_digit = criterion(out_digit, labels_digit)
        loss_person = criterion(out_person, labels_person)
        loss = loss_digit + loss_person  # Gesamtverlust

        # Backpropagation und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Ausgabe des mittleren Epochenverlusts
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


Epoche  1 | Accuracy Digit: 11.39% | Person: 57.78% | Loss: 3.4606
Epoche  2 | Accuracy Digit: 17.50% | Person: 69.72% | Loss: 3.0652
Epoche  3 | Accuracy Digit: 17.78% | Person: 71.67% | Loss: 2.8783
Epoche  4 | Accuracy Digit: 23.06% | Person: 79.44% | Loss: 2.6263
Epoche  5 | Accuracy Digit: 29.17% | Person: 78.06% | Loss: 2.5826
Epoche  6 | Accuracy Digit: 32.50% | Person: 81.67% | Loss: 2.3194
Epoche  7 | Accuracy Digit: 38.06% | Person: 84.44% | Loss: 2.0210
Epoche  8 | Accuracy Digit: 53.33% | Person: 87.22% | Loss: 1.7100
Epoche  9 | Accuracy Digit: 52.78% | Person: 88.89% | Loss: 1.6979
Epoche 10 | Accuracy Digit: 60.56% | Person: 85.28% | Loss: 1.5006
Epoche 11 | Accuracy Digit: 66.94% | Person: 89.72% | Loss: 1.3421
Epoche 12 | Accuracy Digit: 71.67% | Person: 89.72% | Loss: 1.1633
Epoche 13 | Accuracy Digit: 75.83% | Person: 91.11% | Loss: 1.0145
Epoche 14 | Accuracy Digit: 79.44% | Person: 89.72% | Loss: 0.9194
Epoche 15 | Accuracy Digit: 77.50% | Person: 92.50% | Loss: 0.

In [ ]:
# Plots fürs Training, genutzt zum Erstellen einer Grafik für die Dokumentation
combined_train_acc = [0.5 * (d + p) for d, p in zip(train_digit_acc, train_person_acc)]
combined_train_loss = [d + p for d, p in zip(train_digit_loss, train_person_loss)]

plt.figure(figsize=(12, 5))

# Accuracy-Plot
plt.subplot(1, 2, 1)
plt.plot(combined_train_acc, marker="o", label="Train Accuracy", color="green")
plt.title("Gesamt Accuracy")
plt.xlabel("Epoche")
plt.ylabel("Accuracy")
plt.ylim(0, 1.05)
plt.grid(True)
plt.legend()

# Loss-Plot
plt.subplot(1, 2, 2)
plt.plot(combined_train_loss, marker="o", label="Train Loss", color="red")
plt.title("Gesamt Loss")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [20]:
# def evaluate(loader):
#     model.eval()
#     correct_digit = 0
#     correct_person = 0
#     total = 0
#     i=0
#     with torch.no_grad():
#         for images, labels_digit, labels_person in loader:
#             i=i+1
#             images = images.to(device)
#             labels_digit = labels_digit.to(device)
#             labels_person = labels_person.to(device)

#             out_digit, out_person = model(images)
#             pred_digit = out_digit.argmax(dim=1)
#             pred_person = out_person.argmax(dim=1)

#             correct_digit += (pred_digit == labels_digit).sum().item()
#             correct_person += (pred_person == labels_person).sum().item()
#             total += images.size(0)

#     print(f"Digit Accuracy: {correct_digit / total:.2%}")
#     print(f"Person Accuracy: {correct_person / total:.2%}")
#     print(i)

# evaluate(val_loader)


In [ ]:
import os
import numpy as np
from PIL import Image
from collections import defaultdict
from torchvision import transforms

# Pfad zur Evaluationsdaten und Zuordnung der Namen zu IDs
eval_path = "C:/dhbw/6. semester/advanced ml/Zahlenerkennung-main/eval"
person_to_id = {"Tim": 0, "Thanadon": 1, "Nils": 2}
id_to_person = {v: k for k, v in person_to_id.items()}

# Transformation für Eval-Bilder (nur Resize und Tensorumwandlung)
eval_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Statistiken initialisieren (für Ziffern- und Personen-Genauigkeit)
digit_results = defaultdict(lambda: {"digit_total": 0, "digit_correct": 0,
                                     "person_total": 0, "person_correct": 0})
person_results = defaultdict(lambda: {"digit_total": 0, "digit_correct": 0,
                                      "person_total": 0, "person_correct": 0})

# Modell in den Evaluationsmodus setzen und Vorhersagen berechnen
model.eval()
with torch.no_grad():
    for fname in os.listdir(eval_path):
        if not fname.endswith(".npy"):
            continue

        # Bild laden und vorbereiten
        fpath = os.path.join(eval_path, fname)
        img_array = np.load(fpath)
        if img_array.dtype != np.uint8:
            img_array = (img_array * 255 / np.max(img_array)).astype(np.uint8)
        if img_array.ndim > 2:
            img_array = img_array.squeeze()
        img = Image.fromarray(img_array).convert("L")
        img = eval_transform(img).unsqueeze(0).to(device)

        # Labels aus dem Dateinamen extrahieren
        parts = fname.split("_")
        if len(parts) < 3:
            continue
        person_name = parts[0]
        digit_label = int(parts[1])
        person_label = person_to_id.get(person_name, -1)

        # Modellvorhersage
        pred_digit_logits, pred_person_logits = model(img)
        pred_digit = torch.argmax(pred_digit_logits, dim=1).item()
        pred_person = torch.argmax(pred_person_logits, dim=1).item()

        # Statistik pro Ziffer
        digit_results[digit_label]["digit_total"] += 1
        digit_results[digit_label]["person_total"] += 1
        if pred_digit == digit_label:
            digit_results[digit_label]["digit_correct"] += 1
        if pred_person == person_label:
            digit_results[digit_label]["person_correct"] += 1

        # Statistik pro Person
        person_results[person_name]["digit_total"] += 1
        person_results[person_name]["person_total"] += 1
        if pred_digit == digit_label:
            person_results[person_name]["digit_correct"] += 1
        if pred_person == person_label:
            person_results[person_name]["person_correct"] += 1

# Ausgabe pro Ziffer
print("Erkennung pro Ziffer:")
for digit in sorted(digit_results.keys()):
    r = digit_results[digit]
    digit_acc = r["digit_correct"] / r["digit_total"] * 100 if r["digit_total"] > 0 else 0
    person_acc = r["person_correct"] / r["person_total"] * 100 if r["person_total"] > 0 else 0
    print(f"Zahl {digit}:")
    print(f"  → Ziffern-Erkennung: {digit_acc:.2f}% korrekt")
    print(f"  → Personen-Erkennung: {person_acc:.2f}% korrekt")

# Ausgabe pro Person
print("\nErkennung pro Person:")
for person, r in person_results.items():
    digit_acc = r["digit_correct"] / r["digit_total"] * 100 if r["digit_total"] > 0 else 0
    person_acc = r["person_correct"] / r["person_total"] * 100 if r["person_total"] > 0 else 0
    print(f"{person}:")
    print(f"  → Ziffern-Erkennung: {digit_acc:.2f}% korrekt")
    print(f"  → Personen-Erkennung: {person_acc:.2f}% korrekt")

# Gesamtauswertung über alle evaluierten Bilder
total_digits = sum(r["digit_total"] for r in digit_results.values())
total_digit_correct = sum(r["digit_correct"] for r in digit_results.values())
total_person_correct = sum(r["person_correct"] for r in digit_results.values())

print(f"\nGesamt Ziffern-Accuracy: {total_digit_correct / total_digits * 100:.2f}%")
print(f"Gesamt Personen-Accuracy: {total_person_correct / total_digits * 100:.2f}%")


Erkennung pro Ziffer:
Zahl 0:
  → Ziffern-Erkennung: 75.00% korrekt
  → Personen-Erkennung: 91.67% korrekt
Zahl 1:
  → Ziffern-Erkennung: 25.00% korrekt
  → Personen-Erkennung: 91.67% korrekt
Zahl 2:
  → Ziffern-Erkennung: 41.67% korrekt
  → Personen-Erkennung: 100.00% korrekt
Zahl 3:
  → Ziffern-Erkennung: 33.33% korrekt
  → Personen-Erkennung: 83.33% korrekt
Zahl 4:
  → Ziffern-Erkennung: 58.33% korrekt
  → Personen-Erkennung: 100.00% korrekt
Zahl 5:
  → Ziffern-Erkennung: 58.33% korrekt
  → Personen-Erkennung: 83.33% korrekt
Zahl 6:
  → Ziffern-Erkennung: 66.67% korrekt
  → Personen-Erkennung: 100.00% korrekt
Zahl 7:
  → Ziffern-Erkennung: 75.00% korrekt
  → Personen-Erkennung: 100.00% korrekt
Zahl 8:
  → Ziffern-Erkennung: 41.67% korrekt
  → Personen-Erkennung: 66.67% korrekt
Zahl 9:
  → Ziffern-Erkennung: 91.67% korrekt
  → Personen-Erkennung: 100.00% korrekt

Erkennung pro Person:
Nils:
  → Ziffern-Erkennung: 57.50% korrekt
  → Personen-Erkennung: 90.00% korrekt
Thanadon:
  → Zif

In [22]:
# import os

# eval_path = "C:/dhbw/6. semester/advanced ml/Zahlenerkennung-main/eval"
# behaltene_endungen = ("16.npy", "17.npy", "18.npy", "19.npy","36.npy", "37.npy", "38.npy", "39.npy")

# # Alle Dateien im Ordner durchgehen
# for fname in os.listdir(eval_path):
#     if not fname.endswith(behaltene_endungen):
#         fpath = os.path.join(eval_path, fname)
#         if os.path.isfile(fpath):
#             os.remove(fpath)
#             print(f"GELÖSCHT: {fname}")
#         else:
#             print(f"ÜBERSPRUNGEN (kein File): {fname}")
#     else:
#         print(f"BEHALTEN: {fname}")


In [ ]:
#Modell erstellen und speichern
torch.save(model.state_dict(), "model.pth")


In [24]:
# import torch

# state_dict = torch.load("model.pth", map_location="cpu")
# print("\n".join(state_dict.keys()))
